# BYO container with SageMaker

In [1]:
import os
import numpy as np
import sagemaker
import boto3
import time
import math
import json
import re
import pandas as pd
from sagemaker.session import s3_input
from sagemaker.estimator import Estimator

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

client = boto3.client('sagemaker')
region = boto3.Session().region_name
account = boto3.client('sts').get_caller_identity().get('Account')

In [2]:
bucket_name = 'reinvent-aim338'
repo_name = 'reinvent-aim338'
image_tag = 'latest'

In [3]:
output_path = 's3://{}/'.format(bucket_name)
train_path = 's3://{}/cifar10-dataset'.format(bucket_name)
eval_path = 's3://{}/cifar10-dataset'.format(bucket_name)
image = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account, region, repo_name, image_tag)

In [4]:
hyperparams={
    'max_learning_rate':0.4,
    'data_aug_cutout_size':5,
    'batch_size':256,
    'momentum':0.9,
    'batch_norm':"true",
    'epochs':2
}

In [5]:
estimator = Estimator(
    image_name=image,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.p3.2xlarge',
    output_path=output_path,
    base_job_name='reinvent',
    hyperparameters=hyperparams,
    metric_definitions=[{'Name': 'test_acc', 'Regex': 'test_acc:([0-9\\.]+)'}])

estimator.fit({'training': train_path, 'eval': eval_path})

2019-12-05 19:36:21 Starting - Starting the training job...
2019-12-05 19:36:22 Starting - Launching requested ML instances...
2019-12-05 19:37:20 Starting - Preparing the instances for training......
2019-12-05 19:38:18 Downloading - Downloading input data...............
2019-12-05 19:40:24 Training - Downloading the training image......
2019-12-05 19:41:45 Training - Training image download completed. Training in progress....
2019-12-05 19:42:22 Uploading - Uploading generated training model
2019-12-05 19:42:22 Completed - Training job completed
.#0150it [00:00, ?it/s]#015  0%|          | 0/170498071 [00:00<?, ?it/s]#015  0%|          | 40960/170498071 [00:00<10:05, 281475.70it/s]#015  0%|          | 204800/170498071 [00:00<07:39, 370442.72it/s]#015  0%|          | 450560/170498071 [00:00<05:42, 496242.44it/s]#015  1%|1         | 1744896/170498071 [00:00<04:02, 695676.80it/s]#015  2%|1         | 3334144/170498071 [00:00<02:51, 974844.05it/s]#015  4%|3         | 6397952/170498071 [00: